In [ ]:
# connect to milvus db

from langchain_milvus import Milvus
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from pymilvus import MilvusClient
import os
load_dotenv()
MILVUS_URI = os.getenv("MILVUS_URI")
MILVUS_TOKEN = os.getenv("MILVUS_TOKEN")
MILVUS_COLLECTION_NAME = os.getenv("MILVUS_COLLECTION_NAME")

milvus_client = MilvusClient(
    uri=MILVUS_URI,
    token=MILVUS_TOKEN,
    collection_name=MILVUS_COLLECTION_NAME
)

In [90]:
import os
from typing import List
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GEMINI_API_KEY,
)

def get_embedding(text: str) -> List[float]:
    """Generate embedding for the given text using Gemini."""
    return embeddings.embed_query(text)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=GEMINI_API_KEY,
    temperature=0
)

class Grade(BaseModel):
    binary_score: str = Field(description="Relevance score 'yes' or 'no'")

class TranslationWithScoreAndExplanation(BaseModel):
    translation: str = Field(description="The translated text. this should be a single translation not a list of translations , should be the source text")
    score: float = Field(description="Score evaluating the translation quality, between 0 and 100.")
    explanation: str = Field(description="Explanation of the given score for this translation.")
    
class TranslationList(BaseModel):
    items: List[TranslationWithScoreAndExplanation]
    
output_parser = StrOutputParser()

structured_llm_grader = llm.with_structured_output(Grade)
structured_llm_translation_with_score_and_explanation = llm.with_structured_output(TranslationList)

def askgemini(prompt: str,is_boolean:bool ) -> str:
    """Generate a response from Gemini for a given prompt, using LangChain output parser."""
    if is_boolean:
        score = structured_llm_grader.invoke(prompt)
        return score.binary_score=="yes"
    else:
        response = llm.invoke(prompt)
    # Use output parser to ensure string output
    return response


In [102]:
# search a text in milvus

from pymilvus import AnnSearchRequest, RRFRanker



def search_text(milvus_client: MilvusClient, text: str):
    """
    search a text in milvus
    """
    query_embedding=get_embedding(text)
    results = milvus_client.search(
    collection_name=MILVUS_COLLECTION_NAME,
    data=[query_embedding],
    limit=5,
    output_fields=["text"],
    anns_field="dense_vector"
    )
    
    return results

def hybrid_search(milvus_client: MilvusClient, text: str):
    query_embedding=get_embedding(text)
    limit=5
    search_param_1 = {
        "data": [query_embedding],
        "anns_field": "dense_vector",
        "param": {},
        "limit": limit,
    } 
    request_1 = AnnSearchRequest(**search_param_1)
    
    search_param_2 = {
        "data": [query_embedding],
        "anns_field": "dense_vector",
        "param": {"drop_ratio_search": 0.2},
        "limit": limit
    }
    request_2 = AnnSearchRequest(**search_param_2)
    
    
    results = milvus_client.hybrid_search(
    collection_name=MILVUS_COLLECTION_NAME,
    vector=[query_embedding],
    reqs=[request_1,request_2],
    ranker=RRFRanker(),
    limit=limit,
    output_fields=["text","parent_id"]
)
    return results

text_search="abstract"
hybrid_search(milvus_client,text_search)

data: [[{'id': 'oBHDQm3GxAtuWt323H1Py', 'distance': 0.032786883413791656, 'entity': {'parent_id': 'pdxQiFy5eDWhIYcXDSLF0', 'text': 'དཔེར་ན་མིག་སེར་གྱི་ནད་ཀྱིས་བཏབ་པའི་མིག་ཅན་གྱིས་མེད་བཞིན་དུ་དུང་ལ་གསེར་གྱི་རྣམ་པ་ལྟ་བུར་རིག་པ་དེ་བཞིན་དུ་འཁྲུལ་པའི་བག་ཆགས་ཀྱི་དབང་གིས་མེད་ཀྱང་སེར་པོ་ལ་སོགས་པའི་རྣམ་པ་བདེན་པར་མངོན་པར་འདོད་ཅེའོ། །འདིར་ཡང་འདི་འཁྲུལ་པའི་བག་ཆགས་ལ་རག་ལས་པ་ཡིན་ནོ། །'}}, {'id': 'pJMR7iYHGyjDC7g8ljxoF', 'distance': 0.032258063554763794, 'entity': {'parent_id': 'GUkQ8ICEZsVNPdXMTCACF', 'text': 'དེ་ལ་ལུས་ལ་སྤོས་ཀྱིས་བྱུག་པ་ནི་ལུས་ཀྱི་མདོག་མཛེས་པ་དང་དྲི་ཞིམ་པར་བྱ་བའི་ཆེད་དུ་ཨ་ཀ་རུ་དང་འབུ་སུག་ལ་སོགས་པ་དྲི་ཞིམ་པོའི་རྣམ་པ་ལྡེ་གུ་དང་ཕྱེ་མར་བྱས་ཏེ་ལུས་ལ་བསྐུ་ཞིང་གདགས་པར་མི་བྱའོ། །'}}, {'id': 'n7hDT2xPTov2c1pflf8GT', 'distance': 0.0317460335791111, 'entity': {'parent_id': 'nbV7ntgplwYCOkiZCr4yw', 'text': 'དཔེར་ན་རབ་རིབ་ཅན་གྱི་མིག་སྐྱོན་གྱིས་ཉམས་པས་ཟླ་བ་གཉིས་ལ་སོགས་པ་ཡང་དག་པ་མ་ཡིན་པར་མཐོང་བ་བཞིན་ནོ། །'}}, {'id': 'rDMP7yldz6bL4URbFKulV', 'distance': 0.03125, 'entity': {'parent_id': 'Q57a7KtWUy

In [86]:
text_to_translate = "བོད་སྐད་དུ། དབུ་མ་ལ་འཇུག་པ་ཞེས་བྱ་བ།\\nའཇམ་དཔལ་གཞོན་ནུར་གྱུར་པ་ལ་ཕྱག་འཚལ་ལོ། \\nཉན་ཐོས་སངས་རྒྱས་འབྲིང་རྣམས་ཐུབ་དབང་སྐྱེས། །སངས་རྒྱས་བྱང་ཆུབ་སེམས་དཔའ་ལས་འཁྲུངས་ཤིང་། །སྙིང་རྗེའི་སེམས་དང་གཉིས་སུ་མེད་བློ་དང་། །བྱང་ཆུབ་སེམས་ནི་རྒྱལ་སྲས་རྣམས་ཀྱི་རྒྱུ། །"

# Get relevant text from Milvus
relevant_results = hybrid_search(milvus_client, text_to_translate)
relevant_texts = [hit.entity.get("text", "") for hit in relevant_results[0]]

# Prepare context for translation (combine relevant texts)
context = "\n\n".join(relevant_texts)

# Generate translation using Gemini with context from relevant text
def generate_translation_with_context(source_text, context):
    prompt = f"Using the following context, translate the Tibetan text to English.\n\n 1.do not include any helping text only response with the translation\n\n 2. do not include the context in the translation only translate the text source text\n\n Context:\n {context}\n source text:\n {source_text}\n\n Translation:"
    response = askgemini(prompt,False)
    return response

def check_if_context_is_relevant(context,text_to_translate):
    prompt = f"Using the following context, check if the context is related to text in any sense.\n\n 1.do not include any helping text only response with the translation\n\n 2. do not include the context in the translation only translate the text source text\n\n 3. only response with true or false if false then reason why it is not relevant\n\n Context:\n {context}\n source text:\n {text_to_translate}\n\n Translation:"
    response = askgemini(prompt,True)
    return response


print(check_if_context_is_relevant(context,text_to_translate))

# # Generate translations with and without context
translation_with_context = generate_translation_with_context(text_to_translate, context)
translation_without_context = generate_translation_with_context(text_to_translate, "")

# compare the two translations with a score





True


In [92]:


def compare_translations(translation_with_context, translation_without_context):
    prompt = f"Compare the following two translations and give a score between 0 and 100.\n\n source text:\n {text_to_translate}\n\n Translation with context:\n {translation_with_context}\n\n Translation without context:\n {translation_without_context}\n\n Score:"
    response = structured_llm_translation_with_score_and_explanation.invoke(prompt)
    return response

translation_compare=compare_translations(translation_with_context.content, translation_without_context.content)


In [99]:
print(translation_compare.items[0].translation)
print(translation_compare.items[0].score)
print(translation_compare.items[0].explanation)
print('\n\n')
print(translation_compare.items[1].translation)
print(translation_compare.items[1].score)
print(translation_compare.items[1].explanation)


In Tibetan, it is called "Entering the Middle Way."
Homage to Manjushri, the youthful form.
The Hearers, Pratyekabuddhas, and the intermediate Buddhas are born from the King of Shakyas.
The Buddhas and Bodhisattvas arise from them.
The mind of compassion and non-duality,
The Bodhicitta is the stream of the sons of the victors.
95.0
This translation is very accurate and captures the meaning of the original text well. The terminology used is appropriate for the subject matter.



In Tibetan, it is called "Introduction to the Middle Way."
Homage to Manjushri, the youthful one.
Hearers, Pratyekabuddhas, and the middling ones are born from the Sage.
Buddhas and Bodhisattvas arise from the mind of enlightenment.
The mind of compassion and non-duality, and the mind of enlightenment are the lineage of the sons of the victors.
85.0
This translation is also good, but it has a few minor inaccuracies. For example, "Entering the Middle Way" is a more accurate translation of the title than "Introduc